In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# <span style="color:red">ch9.08_chroma→pinecone을 활용한 RAG활용</span>

# 벡터DB : Chroma vs. Pinecone

- chroma : 인메모리 vector DB , 로컬 vector DB
- Pinecone : 클라우드 vector DB
   (https://www.pinecone.io/에서 api key생성 -> .env에 추가(PINECONE_API_KEY등록)

# 0. 패키지 설치

In [3]:
%pip install -q pinecone langchain-pinecone --no-warn-script-location

Note: you may need to restart the kernel to use updated packages.


# 1. knowledge Base 구성을 위한 데이터 생성

In [4]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('data/소득세법(법률)(제21065호)(20260102).docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    #separators=["\n\n","\n"," ",""]
)
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

193

In [5]:
# embedding : OpenAI API  text-embedding-3-large
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
load_dotenv()
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [7]:
%%time
# pinecone vector database 저장
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY")
)
# 데이터를 처음 업로드할 때 
index_name = "tax-index"
# database = PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     index_name=index_name
# )
# 업로드한 벡터DB를 가져올 때
database = PineconeVectorStore(
    embedding=embedding, # 질문을 임베딩하여 유사도 검색
    index_name=index_name
)

CPU times: total: 0 ns
Wall time: 999 μs


# 2. 답변 생성을 위한 Retrieval 

In [8]:
query = "연봉이 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs =database.similarity_search(query,k=3) # 기본k값 : 4

In [12]:
# retrieved_docs[1].page_content
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])
#retrieved_doc

'[전문개정 2009. 12. 31.]\n\n\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31., 2025. 10. 1., 2025. 12. 23.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택

In [14]:
# 답변을 잘 모를때는 아래와 같이 (질문 알때는 위와 같이 retrieved_doc  ) 
retriever = database.as_retriever(
    search_kwargs={"k":3}
)
retrieved_docs=retriever.invoke(query)
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

# 3. 답변 생성

In [16]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4.1-nano")

In [17]:
# upstage에서 받은 20$로 llm을 사용하고 싶다면
from langchain_upstage import ChatUpstage
llm=ChatUpstage(
    model="solar-pro2",
    reasoning_effort="high"   # 느리지만 더 깊게 추론함(low,medium)
)


In [18]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세법 전문가입니다
- [context]를 참고해서 사용자의 질문에 답변해 주세요.
- [context]는 다음과 같아요
{retrieved_doc}
- 질문 : {query}"""

In [19]:
ai_message = llm.invoke(prompt)

In [20]:
print(ai_message.content)

**연봉 5천만원인 직장인의 소득세 계산 (2023년 기준)**

### 1. **근로소득금액 계산**
- **총급여액**: 50,000,000원  
- **근로소득공제** (제47조):  
  - 1,200만원 이하: 70% 공제 → 1,200 × 70% = **840만원**  
  - 1,200만원 초과 ~ 4,600만원 이하: 40% 공제 → (4,600 - 1,200) × 40% = **1,360만원**  
  - 4,600만원 초과 ~ 8,800만원 이하: 15% 공제 → (5,000 - 4,600) × 15% = **60만원**  
  - **총 공제액**: 840 + 1,360 + 60 = **2,260만원**  
  - **단서 적용**: 2,260만원 > 2,000만원 → **2,000만원 공제**  
- **근로소득금액**: 50,000,000 - 20,000,000 = **30,000,000원**

---

### 2. **종합소득과세표준 계산**
- **근로소득금액**: 30,000,000원  
- **기본공제** (본인 1인 기준): 1,500,000원  
- **과세표준**: 30,000,000 - 1,500,000 = **28,500,000원**  
  *(추가 공제 항목(의료비, 보험료 등)이 있을 경우 차감 가능)*

---

### 3. **산출세액 계산 (누진세율 적용)**
| 과세표준 구간         | 세율 | 누진공제 | 계산식                  |
|-----------------------|------|-----------|-------------------------|
| 1,200만원 이하        | 6%   | -         | 12,000,000 × 6% = 720,000원 |
| 1,200만원 ~ 4,600만원 | 15%  | 1,080,000 | (28,500,000 - 12,000,000) × 15% - 1,080,000 |
| **총 산출세액**       |      |    